# Python Basic

JCBioinformatics - 2019 - HZAU


# IO, module, script

1. IO(输入与输出)
    1. 文件 IO
        + 文本文件
        + 标准输入、输出流
        + 压缩文件
    2. 其他
        + 数据库
        + 网络

2. Module
    1. Standard modules
        + os
        + subprocess
        + re(正则表达式)
        + time, datetime
        + 配置文件读写: json, yaml
    2. 自己编写模块
    
3. Script
    + 编写一个简单的 script
    
3. Package
    + 安装 package
    + 制作一个 package
    + 关于 PyPI



# 输入与输出

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/Laptop-hard-drive-exposed.jpg/330px-Laptop-hard-drive-exposed.jpg)

目前为止，我们学习了很多基础的以及如何进行组合与抽象的概念、语法，但除了在做 `print` 的时候，我们的程序像是运行在一个独立的世界当中，并没有与外界产生交互。对于编程实践来说，这显然是不够的，是时候与外面的世界取得连接了。

## 文件读写

在计算机中，信息往往是以文件的形式存储在磁盘上的，而文件系统是操作系统(OS)管理的，我们对文件的访问与操作都需要通过向操作系统发送请求来完成，这也称之为 系统调用(System Call)，Python 对这些操作进行了封装，我们可以很容易的访问文件系统。

### 读取一个文本文件

![](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b3/Illustration_of_a_newline.png/330px-Illustration_of_a_newline.png)

文本文件是能够按照一定的字符编码规则（比如ASCII、UTF-8）将其解释为文本信息的一类文件。它一种非常常用的文件形式，特别是对于类 `Unix` 操作系统来说尤其如此，因为`Unix`的一条设计理念就是一切皆文本，信息储存以及系统、软件配置往往都通过文本文件来实现。


![descriptor](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f8/File_table_and_inode_table.svg/450px-File_table_and_inode_table.svg.png)

首先我们要介绍一个概念叫做 文件句柄(file handle) ，它是 Python 中对于文件的抽象。我们通过文件句柄与操作系统中的文件进行交互，比如我们要读取一个文件：

In [34]:
f = open("./data/example_text.txt")  # 打开一个文件句柄
type(f)

_io.TextIOWrapper

看到我们以默认参数打开的文件句柄的类型为 `_io.TextIOWrapper`，这代表这个文件是以文本模式打开的。

In [35]:
content = f.read() # 将 f 的内容读入内存
print(content) # 将内容打印出来

In computer programming, standard streams are preconnected input and output communication channels[1] between a computer program and its environment when it begins execution.
The three input/output (I/O) connections are called standard input (stdin), standard output (stdout) and standard error (stderr).
Originally I/O happened via a physically connected system console (input via keyboard, output via monitor), but standard streams abstract this.
When a command is executed via an interactive shell, the streams are typically connected to the text terminal on which the shell is running, but can be changed with redirection or a pipeline.
More generally, a child process inherits the standard streams of its parent process.


如果我们再读取一次呢？

In [36]:
f.read()

''

返回了一个空字符串，这是因为 文件描述符 中存在一个 内部状态（或者称之为 `file pointer`） 记录了当前描述符所在的位置，进行文件读取或写入时，都会从这个位置开始。我们可以使用 `f.seek` 重置这个位置，以及使用 `f.tell`查看该位置。

In [37]:
f.seek(0) # 将表述符位置重置为 0,即文件开始处

0

In [38]:
f.tell() # 检查目前所在位置

0

In [39]:
f.read(1) # 读取一个字符

'I'

In [40]:
f.tell()

1

In [41]:
f.readline() # 读取一行文本

'n computer programming, standard streams are preconnected input and output communication channels[1] between a computer program and its environment when it begins execution.\n'

In [42]:
f.tell()

175

#### 关闭句柄
操作系统中文件描述符是一种有限资源，使用完毕后一定要及时关闭。关闭使用 `f.close` 方法：

In [30]:
f.close()

In [31]:
f.closed

True

### with 语句

Python 有一种很好的方式来避免忘记关闭文件句柄的问题，那就是上下文管理器以及`with`语句，比如：

In [33]:
with open("./data/example_text.txt") as f:  # 打开一个文件句柄，保存在变量 f 中
    print(f.read())

f.closed

In computer programming, standard streams are preconnected input and output communication channels[1] between a computer program and its environment when it begins execution.
The three input/output (I/O) connections are called standard input (stdin), standard output (stdout) and standard error (stderr).
Originally I/O happened via a physically connected system console (input via keyboard, output via monitor), but standard streams abstract this.
When a command is executed via an interactive shell, the streams are typically connected to the text terminal on which the shell is running, but can be changed with redirection or a pipeline.
More generally, a child process inherits the standard streams of its parent process.


True

可以看到，在 with 语句块之中，可以正常的使用文件句柄，而且不需要显示的调用 `f.close` 将句柄关闭，当退出语句块之后，f会被自动关闭。

### 写入

我们再来看一下如何将字符串写入文本文件：

In [43]:
contents="""This long string will be writed to
the file: ./data/test_write.txt
"""

with open("./data/test_write.txt", 'w') as f:
    f.write(contents)

这里文件句柄的打开方式，与读取时类似，只是给`open`函数传入了第二个参数 'w'，
这代表以写入模式打开。

让我们来检查一下内容是否有被写入:

In [44]:
! cat ./data/test_write.txt  # IPython/Jupyter 中在输入前加 ! 可以运行 shell 命令

This long string will be writed to
the file: ./data/test_write.txt


### Buffering

实际上，我们每次读写都需要进行系统调用(System call)，也就是说对于磁盘的访问是由操作系统来控制的，每次进行系统调用都会带来一定的额外开销。为了追求效率，会尽量减少系统调用的次数，一般来说会设置一个缓冲区，也叫做`Buffer`，每次读写时当输入或输出的字节数达到一定数量时才会离开Buffer。也就是说被`write`的内容，并不会立即真正被写入文件。比如：

In [49]:
f = open("./data/test_write_2.txt", 'w')
f.write("a new line\n")

11

In [50]:
!cat ./data/test_write_2.txt

可以看到，此时文件中并没有内容。如果我们想立即将 `Buffer` 中的内容写入到文件，可以调用 `f.flush`（刷新）：

In [51]:
f.flush()

In [52]:
!cat ./data/test_write_2.txt

a new line


当然，当文件关闭时，`Buffer`也会被强制刷新。

In [53]:
f.write("another new line\n")
f.close()

In [54]:
!cat ./data/test_write_2.txt

a new line
another new line


`open` 函数还有另一个参数buffering用来控制 buffer 行为，传入 0 可以关闭 buffer（仅在二进制`'b'`模式下允许使用），传入 1 将以行为单位进行 buffer（仅支持文本模式`'t'`下使用）, > 1 时可以指定一个固定大小的 `buffer` 块。

查看默认的 `Buffer` size:

In [48]:
import io
io.DEFAULT_BUFFER_SIZE

8192

### 句柄模式（Mode）

我们打开一个用来写入的句柄时，像`open`函数传入了一个表示写入模式的字符`'w'`，实际上，文件句柄还有其他的打开模式：

```
Character Meaning
--------- ---------------------------------------------------------------
'r'       open for reading (default)
'w'       open for writing, truncating the file first
'x'       create a new file and open it for writing
'a'       open for writing, appending to the end of the file if it exists
'b'       binary mode
't'       text mode (default)
'+'       open a disk file for updating (reading and writing)
'U'       universal newline mode (deprecated)
```

如果我们不像 open 传入相关参数，默认的打开模式为 `'rt'`，即文本读取模式，而传入单个`'w'`时其实默认的是 `'wt'`，即文本写入模式。

#### 追加(append)模式

还有一个有用的模式是追加模式，有时我们向像一个已经存在的文件中写入内容，但是模式 `'w'`将原来存在的文件覆盖掉：

In [57]:
with open("./data/test_write_3.txt", 'w') as f:
    f.write("a new line\n")

with open("./data/test_write_3.txt", 'w') as f:
    f.write("another new line\n")

In [58]:
!cat ./data/test_write_3.txt

another new line


第一次写入的 'a new line' 被覆盖掉了，如果不想让这种情况发生，我们在第二次`open`的时候应该使用追加模式`'a'`：

In [59]:
with open("./data/test_write_3.txt", 'w') as f:
    f.write("a new line\n")

with open("./data/test_write_3.txt", 'a') as f:
    f.write("another new line\n")

In [60]:
!cat ./data/test_write_3.txt

a new line
another new line


### 读写二进制文件

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/7/76/Wikipedia_favicon_hexdump.svg/435px-Wikipedia_favicon_hexdump.svg.png" width=300>

> linux 的 xxd 命令可以用来做 hexdump，查看文件的二进制信息。

二进制文件指的是“非文本”文件，也就是说它其中包含着一些无法被解释为文本的部分。有一些信息形式天然地不适合用文本来表示，比如图像与音乐。而且很多时候二进制文件要比文本文件具有更高的信息密度，占用更少的存储空间。

我们该如何读取一个二进制文件呢？我们尝试读取一个 `png` 图像文件：

In [64]:
with open("./img/sicp_0.png") as f:
    f.read()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x89 in position 0: invalid start byte

可以看到，由于其中包含了无法被 `decode` 成 `utf-8`编码的 `bytes` 程序报错。

面对二进制文件，我们需要使用 `binary mode` 打开文件：

In [70]:
with open("./img/sicp_0.png", 'rb') as f:
    contents = f.read()

print(type(contents))
len(contents)

<class 'bytes'>


647071

上面的代码将图片文件的信息以 `bytes` 的形式读取，赋值给变量 `contents`。

我们还可以将其写入到另一个文件中：

In [71]:
with open("./data/test_write_binary.png", 'wb') as f:  # 写入二进制文件，以 'wb' mode打开
    f.write(contents)

In [73]:
!ls -l ./data/test_write_binary.png

-rw-rw-r-- 1 nanguage nanguage 647071 7月  17 14:05 ./data/test_write_binary.png


### 文本文件也能以二进制形式读取

实际上文本文件在磁盘上也是二进制形式保存的，只不过以文本模式打开的句柄会自动对读入的 `bytes` 进行 `decode` 而已。我们也可以将一个文本文件以二进制形式打开，然后手动 `decode`：

In [77]:
with open("./data/example_text.txt", "rb") as f:
    contents = f.read()

type(contents)

bytes

In [78]:
contents.decode('utf-8')

'In computer programming, standard streams are preconnected input and output communication channels[1] between a computer program and its environment when it begins execution.\nThe three input/output (I/O) connections are called standard input (stdin), standard output (stdout) and standard error (stderr).\nOriginally I/O happened via a physically connected system console (input via keyboard, output via monitor), but standard streams abstract this.\nWhen a command is executed via an interactive shell, the streams are typically connected to the text terminal on which the shell is running, but can be changed with redirection or a pipeline.\nMore generally, a child process inherits the standard streams of its parent process.'

### 压缩文件

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f2/Gzip-Logo.svg/330px-Gzip-Logo.svg.png" width=200>

我们之前提到过，文本文件的信息密度其实并不高，在数据比较大的时候，我们为了节省磁盘空间，会将文件压缩，而压缩文件也是一种二进制文件。

#### 读写 gzip 文件

`Linux`的世界中最常见的压缩文件格式就是以`.gz`为文件结尾的`gzip`压缩文件。Python 内置了用于读写 `gzip` 文件的模块，我们看一下如何使用它：

In [86]:
import gzip

# 打开一个 gzip 句柄，读取内容
with gzip.open("data/example_text.txt.gz") as f:
    contents = f.read()

print(type(contents))
contents

<class 'bytes'>


b'In computer programming, standard streams are preconnected input and output communication channels[1] between a computer program and its environment when it begins execution.\nThe three input/output (I/O) connections are called standard input (stdin), standard output (stdout) and standard error (stderr).\nOriginally I/O happened via a physically connected system console (input via keyboard, output via monitor), but standard streams abstract this.\nWhen a command is executed via an interactive shell, the streams are typically connected to the text terminal on which the shell is running, but can be changed with redirection or a pipeline.\nMore generally, a child process inherits the standard streams of its parent process.'

可以看到，gzip文件句柄的`read`方法返回的是压缩文件内容的`bytes`信息，我们需要对其做 `decode` 才能得到 `str`：

In [87]:
contents.decode('utf-8')

'In computer programming, standard streams are preconnected input and output communication channels[1] between a computer program and its environment when it begins execution.\nThe three input/output (I/O) connections are called standard input (stdin), standard output (stdout) and standard error (stderr).\nOriginally I/O happened via a physically connected system console (input via keyboard, output via monitor), but standard streams abstract this.\nWhen a command is executed via an interactive shell, the streams are typically connected to the text terminal on which the shell is running, but can be changed with redirection or a pipeline.\nMore generally, a child process inherits the standard streams of its parent process.'

相反，当我们想把内容写入一个 gzip 文件的时候需要将 `str` 先 `encode` 成 `bytes`：

In [88]:
str_to_be_write = contents.decode('utf-8')

with gzip.open("./data/test_write_binary.txt.gz", 'w') as f:
    f.write(str_to_be_write.encode('utf-8'))

In [89]:
!zcat data/test_write_binary.txt.gz

In computer programming, standard streams are preconnected input and output communication channels[1] between a computer program and its environment when it begins execution.
The three input/output (I/O) connections are called standard input (stdin), standard output (stdout) and standard error (stderr).
Originally I/O happened via a physically connected system console (input via keyboard, output via monitor), but standard streams abstract this.
When a command is executed via an interactive shell, the streams are typically connected to the text terminal on which the shell is running, but can be changed with redirection or a pipeline.
More generally, a child process inherits the standard streams of its parent process.

#### TextIOWrapper

当我们读写 `gzip` 文件时每次都需要手动去做 `encode`，`decode` 未免显得太麻烦了一些。其实 Python 中的 `io.TextIOWrapper` 可以用来简化这些操作：

In [90]:
from io import TextIOWrapper

with TextIOWrapper(gzip.open("./data/example_text.txt.gz")) as f:
    # 将 gzip 句柄进行了包装, 这时候 f 是一个 TextIOWrapper
    print(type(f))
    contents = f.read() # 不需要再做 decode
    
type(contents)

<class '_io.TextIOWrapper'>


str

对于写入来说也是一样的：

In [91]:
from io import TextIOWrapper

with TextIOWrapper(gzip.open("./data/test_write_binary.txt.gz", 'w')) as f:
    # 将 gzip 句柄进行了包装, 这时候 f 是一个 TextIOWrapper
    print(type(f))
    f.write(contents) # 不需要再做 encode

<class '_io.TextIOWrapper'>


In [92]:
!zcat ./data/test_write_binary.txt.gz

In computer programming, standard streams are preconnected input and output communication channels[1] between a computer program and its environment when it begins execution.
The three input/output (I/O) connections are called standard input (stdin), standard output (stdout) and standard error (stderr).
Originally I/O happened via a physically connected system console (input via keyboard, output via monitor), but standard streams abstract this.
When a command is executed via an interactive shell, the streams are typically connected to the text terminal on which the shell is running, but can be changed with redirection or a pipeline.
More generally, a child process inherits the standard streams of its parent process.

### 其他格式压缩文件

请参考：

+ zip: https://docs.python.org/3/library/zipfile.html
+ xz/lzma: https://docs.python.org/3/library/lzma.html
+ bz2: https://docs.python.org/3/library/bz2.html
+ rar: https://pypi.org/project/rarfile/

## 标准输入、输出、错误流

![stdout](https://upload.wikimedia.org/wikipedia/commons/thumb/7/70/Stdstreams-notitle.svg/330px-Stdstreams-notitle.svg.png)

类 `Unix` 操作系统支持一种称之为 管道(pipeline) 的操作来串连多个 进程(Process) 的输入与输出。
这是通过三个特殊的“文件”来实现的：

1. 标准输入流(stdin)：用来接受另一个进程的输出
2. 标准输出流(stdout)：输出给另一个进程、打印到终端或重定位到文件
3. 标准错误流(stderr)：输出不应该进入输出流的内容，比如报错信息

你可以想象成每一个进程都长着一个耳朵（stdin）和两张嘴（stdout, stderr) 来和用户或者其他进程交流。

在 Python 中，我们可以通过 `sys` 模块取得这三者：

In [95]:
import sys

print(sys.stdin)
print(sys.stdout)
print(sys.stderr)

<_io.TextIOWrapper name='<stdin>' mode='r' encoding='UTF-8'>


我们向 `stdout` 写入一些内容试一试：<del>（话说我们的教程目前为止还没出现传统艺能 [hello world](https://en.wikipedia.org/wiki/%22Hello,_World!%22_program)）</del>

In [101]:
sys.stdout.write("Hello World!")

Hello World!

其实上面的代码和直接使用 `print` 的效果是完全一样的。 

其实 `print` 还有一个参数接受一个文件句柄，只不过默认情况下，这个句柄是 `stdout`而已（
我们其实也可以用 `print` 将内容输出到文件。）

然后试一试 `stderr`：

In [102]:
sys.stderr.write("Hello Err!")

Hello Err!

可以看到，在 notebook 中，从 `stderr` 输出的信息是红色的，因为 `stderr` 一般用来输出错误信息。

### input 函数

`input` 函数可以用来从 `stdin` 读入一个字符串，而且可以附带一条提示。

In [98]:
how_are_you = input("How are you?")

How are you?fine


In [99]:
how_are_you

'fine'

这个函数在编写与用户交互的程序时非常有用。

### 编写一个流处理脚本

利用 Python 读写标准流的能力，我们可以编写像 `grep`, `awk` 那样的流处理工具。

我们目前还没有提到什么是 脚本(Script)，但并不复杂，它只是用来处理一些任务的代码片段而已。我们一般把 Python 脚本保存在一个以 '.py' 结尾的文件里。

这里请你创建一个脚本，命名为 `cut_half.py`，然后将下面的内容写入这个文件：

``` Python
import sys

for line in sys.stdin:
    line = line.strip()
    print(line[:len(line)//2])

```

保存它，然后我们就可以运行了，这里我将它保存在了 'scripts/cut_half.py' 这个文件中，也许你和我不一样。
然后我们运行它：

In [106]:
!cat data/seqs.txt | python scripts/cut_half.py | head -n 5

NTTCCATTTCCAATCTTTTAGTCGGAGAACCAGTAGCTAGCTACTGGTTCTCCGACTAAGGATTGTTTTGGCTAT
NTGCCCAGTCTGAAATGCCTAGTCGGAGAACCAGTAGCTAGCTACTGGTTCTCCGCCTAACTTTCTAGATCTGTG
NCACATTAGAAAGTATTTTAGTCGGAGAACCAGTAGCTAGCTACTGGTTCTCCGACTAATNTACATGTTAGGAGG
NGAAACCATATGATTAGTTAGTCGGAGAACCAGTAGCTAGCTACTGGTTCTCCGACTAACNGAAAAAACACATGC
NTGAATGAAAAGAAAAGTTAGTCGGAGAACCAGTAGCTAGCTACTGGTNCTCCCACTAATNAACACAAAGATTCT


相信经过前面的学习，你应该能够看懂这个脚本所做的事情了：

1. 首先它从标准输入迭代读取 每一行文本
2. 然后用 .strip() 将 这一行末尾的'\n' 去掉
3. 打印 这一行的前一半长度的文本，打印到标准输出

这里我们使用它就像使用 `Unix` 中其他的流处理程式那样接入管道(pipeline)。
我们用它将 `seqs.txt` 中保存的序列截断了一半。

## 数据库IO

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3b/Database_models.jpg/720px-Database_models.jpg" width=400>

尽管文本文件很有用，但文本文件有时候并不能满足我们的数据储存需求，特别是在对大量的数据进行随机访问时，如果没有索引信息，访问的效率将十分糟糕。这时候我们需要使用数据库，Python 也能够满足我们与数据库进行数据读写的需求。但受篇幅限制，这里不再详细描述使用方法，仅给出参考。

1. SQL 数据库：
    + [sqlite3](https://docs.python.org/3.7/library/sqlite3.html): 内置模块
    + [SQLAlchemy](https://www.sqlalchemy.org/): 通用 SQL 操作、ORM
    + [MySQL](https://dev.mysql.com/doc/connector-python/en/connector-python-example-connecting.html)
2. NoSQL 数据库：
    + [redis](https://pypi.org/project/redis/)
    + [MongoDB](https://api.mongodb.com/python/current/index.html)
3. 图(Graph)数据库
    + [neo4j](https://neo4j.com/developer/python/)
    + [redis-graph](https://github.com/RedisGraph/redisgraph-py)

## 网络IO

![network](https://upload.wikimedia.org/wikipedia/en/thumb/a/a6/Internet_layering.svg/375px-Internet_layering.svg.png)

另一种重要的 IO 是网络 IO，这使得我们的程序能够通过互联网与另一台计算机上的进程之间进行通讯。同数据库一样，网络编程也是一个很大的话题，
有兴趣可以自己学习，此处仅仅给出一些参考：

我们知道互联网通讯协议(Protocol)是一个层次化的体系，相应的， IO 可以发生在这些不同层协议的基础上。
一般Python经常用于以传输层的 `TCP`, `UDP` 以及应用层 `HTTP` 协议为基础的网络 IO。

1. Socket 编程：标准包 [socket](https://docs.python.org/3/library/socket.html)
2. HTTP 编程：
    + 服务端：
        + [flask](https://palletsprojects.com/p/flask/)
        + [django](https://www.djangoproject.com/)
    + 客户端：
        + [urllib](https://docs.python.org/3/library/urllib.html)
        + [requests](https://2.python-requests.org/en/master/)




# 模块(Module)

在 Python 中一个 `module` 可以认为就是一个 Python 文件(`.py`文件)，我们可以把各种对象放置在一个 `module` 中，
在我们需要时从 `module` 中 导入(`import`） 到当前的环境中。

## 创建一个 module

我们可以创建一个 Python 模块，比如 `seq.py`, 在里面放入一些用于处理 DNA序列 信息的代码：

``` Python
"""
part1-basic/seq.py
"""

def count_GC_ratio(seq):
    n_gc = 0
    for b in seq:
        B = b.upper()
        if B == 'G' or B == 'C':
            n_gc += 1
    return n_gc / len(seq)

def reverse_complement(seq):
    table = {'c':'g', 'g':'c', 'a':'t', 't':'a', 'n':'n'}
    table_upper = {k.upper():v.upper() for k,v in table.items()}
    table.update(table_upper)
    return "".join(list(map(lambda b: table[b], seq[::-1])))
    
```

我们可以导入(`import`)这个模块，然后使用它里面的对象：

In [118]:
import seq

In [119]:
s = "AAAGGTTCCG"

In [120]:
seq.reverse_complement(s)

'CGGAACCTTT'

In [121]:
seq.count_GC_ratio(s)

0.5

当我们 `import` 一个模块时，模块会出现在当前的环境中，模块中的对象通过 `模块名.对象名` 的方法访问。

还有一种导入方式可以直接将 模块中的对象 导入当前环境：

In [122]:
from seq import reverse_complement

In [123]:
# 现在可以直接调用 rc 函数了

reverse_complement(s)

'CGGAACCTTT'

也可以直接导入一个模块中的所有变量：

In [124]:
from seq import *

In [125]:
count_GC_ratio(s)

0.5

## 标准模块 Standard module

Python 内置了丰富的标准模块，其中有一些使用 `C` 编写的，可以提供操作系统级别的功能（sys, os, ...），还有一些提供了方便的特性（functools, itertools, ...），以及一些特定用途的工具（re, json, ...）。

### os - 操作系统接口模块

`os` 模块提供了很多用于与操作系统交互的功能，比如说对文件系统中文件与目录的访问，获取、修改文件权限，切换工作目录等等，
而且 `os` 模块允许我们直接调用 `shell` 命令，有了 `os` 我们就可以使用 Python 代替 `bash` 编写系统管理脚本了。

In [126]:
import os

获取当前工作路径：

In [128]:
os.getcwd()

'/home/nanguage/Dropbox/workspace/JCBioinformatics-2019-Python/part1-basic'

获取目录下的所有文件以及目录：

In [130]:
os.listdir("./")

['__pycache__',
 '.ipynb_checkpoints',
 'scripts',
 'part1c - IO, Module.ipynb',
 'data',
 'img',
 'part1a - Basic.ipynb',
 'part1b-Composition and Abstraction.ipynb',
 'README.md',
 'seq.py']

获取一个文件或目录的绝对路径

In [133]:
os.path.abspath("./img")

'/home/nanguage/Dropbox/workspace/JCBioinformatics-2019-Python/part1-basic/img'

从一个绝对路径中得到文件的目录：

In [138]:
path = os.path.abspath("./part1a - Basic.ipynb")
print(path)
print(os.path.dirname(path))

/home/nanguage/Dropbox/workspace/JCBioinformatics-2019-Python/part1-basic/part1a - Basic.ipynb
/home/nanguage/Dropbox/workspace/JCBioinformatics-2019-Python/part1-basic


从一个绝对路径中得到文件的名字：

In [142]:
path = os.path.abspath("./part1a - Basic.ipynb")
print(path)
print(os.path.basename(path))

/home/nanguage/Dropbox/workspace/JCBioinformatics-2019-Python/part1-basic/part1a - Basic.ipynb
part1a - Basic.ipynb


还有一个函数可以直接切分开目录与文件名：

In [143]:
path = os.path.abspath("./part1a - Basic.ipynb")
os.path.split(path)

('/home/nanguage/Dropbox/workspace/JCBioinformatics-2019-Python/part1-basic',
 'part1a - Basic.ipynb')

得到文件名后缀：

In [144]:
os.path.splitext("./part1a - Basic.ipynb")

('./part1a - Basic', '.ipynb')

判断一个路径或文件是否存在：

In [145]:
os.path.exists("./part1a - Basic.ipynb")

True

In [147]:
os.path.exists("./img/")

True

In [146]:
os.path.exists("./this_does_not_exist.txt")

False

获取环境变量的值：

In [150]:
os.environ['HOME']

'/home/nanguage'

调用 shell 命令：

In [153]:
os.system("touch data/test_call.txt")  # 调用 shell 命令创建一个文件

0

返回值为 0 代表调用成功

In [156]:
"test_call.txt" in os.listdir("data/")

True

## subprocess

上面我们看到了使用 `os.system` 调用一个 shell 命令，实际上 Python 中有一个更好用的标准模块用于来做这个事情。

In [157]:
import subprocess

In [158]:
subprocess.call(["touch", "data/test_subprocess.txt"])

0

不同于 `os.system`, `subprocess` 默认需要将参数按照空格分开，拆成一个 `list` 再传入，如果不这样做就会报错：

In [163]:
try:
    subprocess.call("touch data/test_subprocess.txt")
except Exception as e:  # 报错信息太长了，我处理一下
    print(e)

[Errno 2] No such file or directory: 'touch data/test_subprocess.txt': 'touch data/test_subprocess.txt'


这么做的目的是在编写软件时防止命令行注入攻击，试想假如你通过调用 shell 来完成一些任务，但 shell 的参数来自于一个外来的输入，这时候如果有坏人在输入中偷偷埋藏了一个 `;rm -rf ./*;` 会有多么惨烈的结果。 

如果你能确定这种情况不会发生，也可以加 `shell=True` 参数来关闭这个功能：

In [161]:
subprocess.call("echo 1", shell=True)

0

我们发现，上面的调用都没有产生输出，如果我们像获取 shell 调用所产生的输出怎么办呢?

In [170]:
from subprocess import PIPE
from io import TextIOWrapper

p = subprocess.Popen(["ls"], stdout=PIPE) # 创建一个进程，设置 stdout=PIPE 捕获 stdout

for line in TextIOWrapper(p.stdout):  # 使用 TextIOWrapper 包装 p.stdout 获取字符串输出
    print(line, end="")

data
img
part1a - Basic.ipynb
part1b-Composition and Abstraction.ipynb
part1c - IO, Module.ipynb
__pycache__
README.md
scripts
seq.py


## 正则表达式

![](https://upload.wikimedia.org/wikipedia/commons/thumb/9/94/DFA_example_multiplies_of_3.svg/375px-DFA_example_multiplies_of_3.svg.png)

在计算机科学中，正则表达式(regular expression)是一种描述3类文法（即正则文法）的语言，它的描述能力等同于一个确定状态有限自动机（deterministic finite automaton）。在实际的编程中它常被用来做字符串匹配，在文本处理之中非常有用，所以你能在各种用于
文本处理的 `Linux` 命令行中见到它的身影。
而生物信息编程很大一部分就是文本处理，所以很有必要详细了解一下正则表达式的使用方法。

在 Python 中，正则表达式位于标准模块 `re` 当中。

In [171]:
import re

首先，最简单的，字符串本身就是一个正则表达式，它匹配它自身：

In [176]:
pattern = "AATT"
re.match(pattern, "AATT")

<_sre.SRE_Match object; span=(0, 4), match='AATT'>

这里我们调用了 `re.match` 函数，它接受一个 正则表达式，与一个待匹配的字符串，如果该字符串的**开始位置**能够找到这个正则表达式的**任意个匹配**，返回匹配到的结果，若找不到这样的匹配则不返回值。

然后，特殊字符 '.' 用来表示匹配所有可能的字符：

In [177]:
pattern = "C.G"  # 匹配左边是 C 右边是 G 中间是任意字符的字符串
re.match(pattern, "CAG") # 符合 pattern 可以匹配

<_sre.SRE_Match object; span=(0, 3), match='CAG'>

In [179]:
re.match(pattern, "ACG") # 不符合 pattern 不能匹配

除了 '.' 这样代表字符范围的特殊字符，还有一种特殊字符用来代表 重复次数 这样的特殊字符称之为 **重复修饰符**。

比如 '*' 连接在一个字符的后面代表匹配这个字符串 0-无限 次：

In [180]:
pattern = ".*CG.*"  # 匹配确定一个序列中是否存在 CpG island

re.match(pattern, "AAAAACGTTT")

<_sre.SRE_Match object; span=(0, 10), match='AAAAACGTTT'>

In [182]:
re.match(pattern, "AAAATTTTT")  # 匹配不到

我想你已经注意到了，正则表达式'.*' 可以匹配任何字符串

一个与'*'相似的重复修饰符是'+'，表示匹配前一个字符 1-无限 次

In [184]:
re.match("A+TTT", "AAATTT")

<_sre.SRE_Match object; span=(0, 6), match='AAATTT'>

重复修饰符'?'表示匹配前一个字符 0 或 1 次：

In [197]:
pattern = "AT?"
print(re.match(pattern, "A") is not None)
print(re.match(pattern, "AA") is not None)
print(re.match(pattern, "AT") is not None)
print(re.match(pattern, "TA") is not None)

True
True
True
False


匹配次数也是可以指定的，使用修饰符 `{m,n}` 匹配前一个字符 m-n 次：

In [199]:
pattern = "G{2,5}" # 匹配 G 2-5 次
print(re.match(pattern, "G") is not None)
print(re.match(pattern, "GG") is not None)
print(re.match(pattern, "GGG") is not None)
print(re.match(pattern, "AGG") is not None)

False
True
True
False


#### 非贪婪匹配

我们上面提到的重复修饰符`*`,`+`,`?`都是贪婪(greed)匹配的，也就是说它会匹配尽可能多的字符：

In [202]:
m = re.match("A.*T", "ATAAAT")
print(m)
print(m.span())

<_sre.SRE_Match object; span=(0, 6), match='ATAAAT'>
(0, 6)


如果想要非贪婪的匹配，可以使用他们的非贪婪版本：`*?`, `+?`, `??`

In [203]:
m = re.match("A.*?T", "ATAAAT")
print(m)
print(m.span())

<_sre.SRE_Match object; span=(0, 2), match='AT'>
(0, 2)


### 其他特殊字符

还有一些特殊字符是表示 特殊位置(`^`,`$`)与字符范围(`[0-9]`, `\S`)的，更多请参阅[文档](https://docs.python.org/3/library/re.html#module-re)

### 模式搜索

上面，我们使用的是 `match` 函数对一个字符串的开始位置进行匹配，但更多的时候，我们需要的是模式搜索。模式搜索也就是说在一个字符串中从头搜到尾部，查看是否有匹配，有就返回，在 Python 中对应 `re.search` 函数。

In [204]:
re.search("AATT", "AAATTTTAAATTTT")

<_sre.SRE_Match object; span=(1, 5), match='AATT'>

查找所有匹配

In [206]:
re.findall("AATT", "AAATTTTAAATTTT")

['AATT', 'AATT']

捕获组

In [208]:
re.findall("A(AT)T", "AAATTTTAAATTTT")

['AT', 'AT']